In [239]:
import pygame
from pygame.locals import *
import pygame.sprite
import random

In [240]:
seaColor = (12, 15, 95)
hitColor = (211, 0, 0)
missColor = (230, 230, 230)
width = 854
height = 427

In [241]:
class Sprite(pygame.sprite.Sprite):
    def __init__(self, width, height):
        super().__init__()
        
        self.image = pygame.Surface((width, height))
        self.image.fill(seaColor)
        self.rect = self.image.get_rect()
        self.empty = False
        
        

In [242]:
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

In [243]:
enemy_sprites = pygame.sprite.Group()
ally_sprites = pygame.sprite.Group()

In [244]:
spacing = 3
totSpacing = spacing * 11
spriteSize = 30
startBufferX = 50
startBufferY = 50

In [245]:
enemy_sprite_positions = [
    ((spriteSize + spacing) * col + spacing + startBufferX, (spriteSize + spacing) * row + spacing + startBufferY)
    for row in range(10)
    for col in range(10)
]

ally_sprite_positions = [
    ((spriteSize + spacing) * col + spacing + startBufferX + height, (spriteSize + spacing) * row + spacing + startBufferY)
    for row in range(10)
    for col in range(10)
]

In [246]:
for position in enemy_sprite_positions:
    sprite = Sprite(spriteSize, spriteSize)
    sprite.image = pygame.Surface((spriteSize, spriteSize))
    sprite.image.fill(seaColor) 
    sprite.rect = sprite.image.get_rect()
    sprite.rect.topleft = position
    enemy_sprites.add(sprite)
    
for position in ally_sprite_positions:
    sprite = Sprite(spriteSize, spriteSize)
    sprite.image = pygame.Surface((spriteSize, spriteSize))
    sprite.image.fill(seaColor) 
    sprite.rect = sprite.image.get_rect()
    sprite.rect.topleft = position
    ally_sprites.add(sprite)

In [247]:
def verticalShip(shipSize, sprites):
    found = False
    valid = True
    while not found:    # while ship position is not found
        randomNum = random.randint(0, 99)
        if (randomNum + (shipSize * 10) <= 99):     # if ship position is within grid size
            j = randomNum
            valid = True
            while (j < randomNum + (shipSize * 10)):
                if (sprites.sprites()[j].empty == True):
                    valid = False
                j += 10
            if valid:
                j = randomNum
                while (j < randomNum + (shipSize * 10)):
                    sprites.sprites()[j].empty = True
                    j += 10    
                found = True         

In [248]:
def horizontalShip(shipSize, sprites):
    found = False
    valid = True
    while not found:    # while ship position is not found
        randomNum = random.randint(0, 99)
        if ((randomNum + shipSize) % 10 > randomNum % 10) :
            j = randomNum
            valid = True
            while (j < randomNum + shipSize):
                if (sprites.sprites()[j].empty == True):
                    valid = False
                j += 1
            if valid:
                j = randomNum
                while (j < randomNum + shipSize):
                    sprites.sprites()[j].empty = True
                    j += 1    
                found = True

In [249]:
sizes = [5, 4, 3, 3, 2]
for size in sizes:  #for each size ship
    randOrientation = random.randint(1,2)
    if (randOrientation == 1):
        verticalShip(size, enemy_sprites)
    else:
        horizontalShip(size, enemy_sprites)
            

In [250]:
def main():
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONUP:
                pos = pygame.mouse.get_pos()
                clicked_sprites = [sprite for sprite in enemy_sprites if sprite.rect.collidepoint(pos)]
                for sprite in clicked_sprites:
                    sprite_width, sprite_height = sprite.image.get_size()
                    local_pos = pos[0] - sprite.rect.left, pos[1] - sprite.rect.top
                    if (0 <= local_pos[0] < sprite_width) and (0 <= local_pos[1] < sprite_height):
                        if (sprite.image.get_at(local_pos) == seaColor and sprite.empty) :
                            sprite.image.fill(hitColor)
                        elif (sprite.image.get_at(local_pos) == seaColor):
                            sprite.image.fill(missColor)

        enemy_sprites.update()
        ally_sprites.update()
        
        screen.fill((0, 0, 0))
        enemy_sprites.draw(screen)
        ally_sprites.draw(screen)

        pygame.display.flip()
        clock.tick(60)

In [251]:
if __name__ == "__main__":
    pygame.init()
    main()
    
pygame.quit()